In [26]:
!pip install -q datasets transformers sentence_transformers faiss-gpu

In [29]:
from google.colab import userdata
hf_token=userdata.get('huggingface')

In [3]:
from datasets import load_dataset
dataset=load_dataset('cnn_dailymail','2.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [9]:
dataset['train']

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [7]:
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
tokenizer=AutoTokenizer .from_pretrained('NousResearch/Llama-2-7b-hf')

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
input_count=[len(tokenizer.tokenize(ex['article'])) for ex in dataset['train']]


In [15]:
output_count=[len(tokenizer.tokenize(ex['highlights'])) for ex in dataset['train']]
combined_count=[input+output for input,output in zip(input_count,output_count)]

In [19]:
valid_index=[i for i,count in enumerate(combined_count) if count<=2048]

In [25]:
dataset['train']=dataset['train'].select(valid_index)
token_count=[combined_count[i] for i in valid_index]

In [28]:
from sentence_transformers import SentenceTransformer
import faiss
from datasets import Dataset,DatasetDict
from tqdm.autonotebook import tqdm
import numpy as np

def deduplicate_dataset(dataset:Dataset,model:str,threshold:float):
  sentence_model=SentenceTransformer(model)
  output=[ex['highlights'] for ex in dataset['train']]

  print('converting to embeddings...')
  embed=sentence_model.encode(output,show_progress_bar=True)
  normalized_embed=embed/np.linalg.norm(embed,axis=1,keepdims= True)
  dim=embed.shape[1]
  index=faiss.IndexFlatIP(dim)
  index.add(normalized_embed)

  print('creating nondupe dataset')
  D,I=index.search(normalized_embed,k=2)
  to_keep=[]
  for i in tqdm(range(len(embed)),desc="Filtering"):
    if D[i,1]<threshold:
      to_keep.append(i)

  dataset=dataset['train'].select(to_keep)
  return DatasetDict({'train':dataset})

non_duped_dataset=deduplicate_dataset(dataset,"thenlper/gte-large",0.95)



.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

converting to embeddings...


Batches:   0%|          | 0/8483 [00:00<?, ?it/s]

creating nondupe dataset


Filtering:   0%|          | 0/271435 [00:00<?, ?it/s]

In [33]:
print(non_duped_dataset['train'])

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 247126
})


In [ ]:

k=10000
instruction_token_counts = [len(tokenizer.tokenize(example["article"])) for example in non_duped_dataset['train']]
output_token_counts = [len(tokenizer.tokenize(example["highlights"])) for example in non_duped_dataset['train']]
combined_token_counts = [instruction + output for instruction, output in zip(instruction_token_counts, output_token_counts)]


In [52]:
def top_k(dataset,token_counts,k):
  sorted_index=sorted(range(len(token_counts)),key=lambda i:token_counts[i],reverse=True)
  top_k_index=sorted_index[:k]

  top_k_data={
      "article":[dataset['train'][i]["article"] for i in top_k_index],
      "highlights":[dataset['train'][i]["highlights"] for i in top_k_index],
  }

  return Dataset.from_dict(top_k_data)

In [53]:
top_k_dataset = top_k(non_duped_dataset, combined_token_counts, k)
dataset = DatasetDict({"train": top_k_dataset})

In [54]:
dataset["train"].to_pandas()

,article,highlights
0,(CNN) -- Heavy snow pummeled much of the East ...,NEW: Many Interstate highways closed in Pennsy...
1,(CNN) -- Nelson Mandela: Two decades before he...,"20 years ago today, Nelson Mandela was release..."
2,(The Frisky) -- When you're in a relationship ...,A list of 13 romantic spots and ideas that'll ...
3,"(RealSimple.com) -- ""I am leaving,"" I murmured...",Judith Newman lists the many ways things can g...
4,Washington (CNN) -- A winter storm slammed the...,"Promises kept on Iraq withdrawal, reducing rep..."
...,...,...
9995,"By . Jessica Jerreat . PUBLISHED: . 12:37 EST,...",Day of action calls for civil liberty charges ...
9996,"By . Kieran Corcoran . PUBLISHED: . 16:21 EST,...",Garish red and silver Rolls-Royce Phantom spot...
9997,By . Associated Press . and Ryan Gorman . PUBL...,"Ismael 'El Mayo' Zambada, 66, grew up with for..."
9998,"By . Bianca London . PUBLISHED: . 06:00 EST, 1...",Star swears by simple beauty regime of E45 cre...


In [55]:
def chat_template(example):
    example["article"] = f"### SUMMARIZE THE FOLLOWING ARTICLE:\n{example['article']}\n\n### SUMMARY:\n"
    return example
###have to come up with a better prompt
dataset = dataset.map(chat_template)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [56]:
dataset['train'][0]['article']

'### SUMMARIZE THE FOLLOWING ARTICLE:\n(CNN) -- Heavy snow pummeled much of the East Coast on Wednesday, battering states for the second time in a week and forcing many people to stay home from school and jobs. Several cities had record snowfalls. The storm canceled or delayed flights in several cities, kept federal workers home for a third straight day in Washington, and taxed local government budgets as cities and counties scrambled to pay for snow removal, overtime, salt, supplies and equipment. In Washington, the snow was falling at a rate of 2 inches per hour at one point in the afternoon, CNN meteorologist Jacqui Jeras said. Forecasters predicted that Washington winds would gust to 50 mph overnight. A blizzard warning was to be in effect in the Washington area until 7 p.m., the National Weather Service said. Blowing snow caused such poor visibility at midday that snowplows temporarily parked by the side of the road, authorities said. Forecasters predicted that the storm would dum

In [57]:
dataset['train'][0]['highlights']

"NEW: Many Interstate highways closed in Pennsylvania .\nTwo main Washington airports aren't set to reopen until Thursday morning .\nWashington area, New York, Philadelphia under blizzard warning .\nThis winter is snowiest on record for several cities, National Weather Service says ."

In [63]:
dataset.push_to_hub("rjindal/rohit_cnn", token=hf_token)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rjindal/Jindal_cnn/commit/4a8f4bb77eb723bd8e79173186706f840cd36ef6', commit_message='Upload dataset', commit_description='', oid='4a8f4bb77eb723bd8e79173186706f840cd36ef6', pr_url=None, pr_revision=None, pr_num=None)